In [1]:
# rmbranto 2021/08 - original
# rmbranto 2021/10/12 - rewrite

# Demonstrate R programming lanquage 'spocc', 'scrubr' and 'ggplot2' packages to retrieve and combine
# selected invasive species occurrence data from multiple source, see also: 
# https://ocean.si.edu/ocean-life/5-invasive-species-you-should-know 
# https://docs.ropensci.org/spocc/

options(stringsAsFactors = FALSE)

library(spocc)
library(rinat)
library(sparsesvd)
library(scrubr)
library(stringr)
library(plyr)
library(sf)

#load(file='invasives.rda')
#objects()

########################################################################################
########################################################################################
# setup extraction run ...

fName<-'invasives'
fao.loc="geobase/World_Fao_Zones.shp"
eez.loc="geobase/EEZ_Land_v3_202030.shp"
species.list<-read.csv(paste(fName,'_list.csv',sep=''))
qNames=species.list$Names
prov.list=read.csv('prov_list.csv')
pNames=prov.list$prov

limit=9999
limit=99

extract=FALSE
extract=TRUE

########################################################################################
########################################################################################
# extract data ...

if(extract){
do.extract<-function(qName,pNames,limit){
    df1<-data.frame(qName=qName,
                    occ2df(occ(query = qName, 
                               from=pNames[!(pNames %in% c('UNIQUE','DUPS','inat'))], 
                               limit=limit, 
                               has_coords = TRUE)))
    df2<-get_inat_obs(
            query=qName,quality="research",
            geo=TRUE,
            maxresults=limit)    
    df2<-data.frame(qName=qName,
              name=df2$scientific_name,
              longitude=df2$longitude,
              latitude=df2$latitude,
              prov='inat',
              date=df2$datetime,
              key=df2$id)
    return(rbind(df1,df2))
}
df<-NULL
for(qName in qNames){
    cat(qName,'...\n')
    df<-rbind(df,do.extract(qName,pNames,limit))
    }
df.raw<-df
}

cat('\n# df.raw counts by species ...\n\n')
table(df.raw$qName,df.raw$prov)%>%
rbind(.,total=apply(.,2,sum,na.rm=TRUE))%>%
cbind(.,total=apply(.,1,sum,na.rm=TRUE))%>%
print(.,na.print='.')   

########################################################################################
########################################################################################
# scrub data

df<-df.raw
names(df)[1]<-'species'

# fix coordinates

df$latitude<-round(as.numeric(df$latitude),12)
df$longitude<-round(as.numeric(df$longitude),12)

df<-dframe(df) %>%
    coord_impossible() %>%
    coord_incomplete() %>%
    coord_unlikely()

# fix dates ...
df<-df[!is.na(df$date),]
df<-df[as.character(df$date)>'1111-11-11',]

df<-df[df$date<=Sys.Date(),]

df.clean<-df
nrow(df.clean)
range(df.clean$date)

cat('\n# df.clean counts by species & prov ...\n\n')
table(df.clean$species,df.clean$prov)%>%
rbind(.,col.totals=apply(.,2,sum,na.rm=TRUE))%>%
cbind(.,ALL=apply(.,1,sum,na.rm=TRUE))%>%
print(.,na.print='.')   

cat('\n# df.clean occurrrence counts by species & provs and by ALL, UNIQUE & DUPS ...\n\n')
table(df.clean$species,df.clean$prov)%>%
cbind(.,ALL=apply(.,1,sum,na.rm=TRUE))%>%
cbind(.,UNIQUE=table(count(df.clean,c('species','longitude','latitude','date'))$species))%>%
cbind(.,DUPS=.[,'ALL']-.[,'UNIQUE'])%>%
rbind(.,col.totals=apply(.,2,sum,na.rm=TRUE))%>%
print(.,na.print='.')   

########################################################################################
########################################################################################
# create df.UNIQUE ...
cat('\n# df.unique...\n\n')
df.UNIQUE<-count(df.clean,c('species','longitude','latitude','date'))
names(df.UNIQUE)[5]<-'DUPS'
df.UNIQUE$id=seq.int(nrow(df.UNIQUE))
cat('UNIQUE=',nrow(df.UNIQUE),'\n')
head(df.UNIQUE)

#create df.ALL ...
cat('\n# df.ALL...\n\n')
df.ALL=merge(df.clean,df.UNIQUE,by=c(c('species','longitude','latitude','date')))
cat('ALL=',nrow(df.ALL),'\n')
cat('UNIQUE=',nrow(count(df.ALL$id)),'\n')
cat('DUPS=',length(df.ALL$DUPS[df.ALL$DUPS>1])-nrow(count(df.ALL$id[df.ALL$DUPS>1])))
head(df.ALL)

# create df.DUPS ...
cat('\n# df.DUPS...\n\n')
df.DUPS=count(df.ALL[df.ALL$DUP>1,],c('species','longitude','latitude','date','id'))
names(df.DUPS)[6]='DUPS'
df.DUPS=df.DUPS[,c('species','longitude','latitude','date','DUPS','id')]
cat('nrows=',nrow(df.DUPS),'\n')
cat('DUPS=',sum(df.DUPS$DUPS)-nrow(df.DUPS),'\n')
head(df.DUPS)

########################################################################################
########################################################################################
# eez and fao indexing

cat('\n# unique PTS...\n\n')

fao.shp=st_read(fao.loc,quiet=T)
eez.shp=st_read(eez.loc,quiet=T)

prov.pts<-count(df.ALL,c('latitude','longitude'))[,1:2]
nrow(prov.pts)

cat('\n# EEZ...\n\n')

eez.pts<-st_as_sf(prov.pts,coords = c('longitude','latitude'), crs = st_crs(eez.shp))
eez.intersect<-data.frame(st_intersects(eez.pts, eez.shp))
nrow(eez.intersect)

prov.pts$eez='UNK'
for(i in 1:nrow(eez.intersect)){
    prov.pts$eez[eez.intersect$row.id[i]]<-eez.shp$ISO_TER1[eez.intersect$col.id[i]]    
}

cat('\n# FAO...\n\n')

fao.pts<-st_as_sf(prov.pts,coords = c('longitude','latitude'), crs = st_crs(fao.shp))
fao.nearest<-data.frame(st_nearest_feature(fao.pts, fao.shp))
nrow(fao.nearest)

prov.pts$fao='99'
for(i in 1:nrow(fao.nearest)){
    prov.pts$fao[i]<-fao.shp$zone[fao.nearest[i,1]]    
}

cat('\n# df.ALL...\n\n')

df.ALL<-merge(df.ALL,prov.pts,by=c('longitude','latitude'))#[,c(3:9,1:2,10:11)]

cat('nrows= ',nrow(df.ALL),'\n')
head(df.ALL,5)
table(df.ALL$fao)[rev(order(table(df.ALL$fao)))]
table(df.ALL$eez)[rev(order(table(df.ALL$eez)))][1:26]

Registered S3 method overwritten by 'hoardr':
  method           from
  print.cache_info httr
Linking to GEOS 3.9.1, GDAL 3.3.0, PROJ 8.0.0


Carcinus maenas ...
Caulerpa taxifolia ...
Codium fragile ...
Dreissena polymorpha ...


Warning message in foo_ala(sources, x, y, s, z, hc, d, w, ds$ala):
“No records returned in ALA for Dreissena polymorpha”

Mnemiopsis leidyi ...


Warning message in foo_ala(sources, x, y, s, z, hc, d, w, ds$ala):
“No records returned in ALA for Mnemiopsis leidyi”

...

[1] 263


# df.ALL...

nrows=  291 


,longitude,latitude,species,date,name,prov,key,DUPS,id,eez,fao
,<dbl>,<dbl>,<chr>,<date>,<chr>,<chr>,<chr>,<int>,<int>,<chr>,<chr>
1,-0.02023197,49.31939,Carcinus maenas,2021-10-12,Carcinus maenas,inat,98006821,1,21,FRA,27
2,-117.24514700,32.66790,Codium fragile,2021-01-12,Codium fragile (Suringar) Hariot,gbif,3031703955,1,101,USA,77
3,-117.27236000,32.85115,Codium fragile,1968-06-06,codium fragile,idigbio,0106de2f-d1a3-45f2-b9dc-ac876bcd2acb,1,100,USA,77
4,-118.33333300,33.35000,Codium fragile,1948-03-25,Codium fragile,ala,fbfc311c-1ce4-4724-80be-1e80a946e4e0,1,99,USA,77
5,-118.37580500,33.73920,Codium fragile,2021-01-15,Codium fragile (Suringar) Hariot,gbif,3031702977,1,98,USA,77



21 31 37 27 77 81 57 67 71 61 51 41 47 
59 47 38 37 22 21 21 16 13  6  6  3  2 


USA CAN AUS ITA UKR NLD NZL GBR IND DEU MEX DNK JPN IDN GEO EST SWE RUS NOR NCL 
 84  36  33  15   9   9   7   7   6   6   4   4   3   3   3   3   2   2   2   2 
NAM MKD LKA FRA CYM CHE 
  2   2   2   2   2   2 

In [7]:
########################################################################################
########################################################################################
# get taxon.ids for each provider and species 
#

prov.keys<-data.frame(species=species.list$Names)

for(my.prov in c('ala','bison','gbif','idigbio','inat','obis')){

spec.keys<-NULL

for (my.species in species.list$Names){
    
    an.error.occured <- FALSE
    
    tryCatch(
        
    { 
      if(my.prov=='ala'){
        result=as.character(
        as.ala(occ(query=my.species,from='ala',limit=1))[[1]]$processed$classification$left
        )
      }
      if(my.prov=='bison'){
        result=as.character(
        as.bison(occ(query=my.species,from='bison',limit=1))[[1]]$points$ITIStsn
        )
      }
      if(my.prov=='gbif'){
        result=as.character(
        as.gbif(occ(query=my.species,from='gbif',limit=1, has_coords = TRUE))[[1]][[1]]$hierarchy$key[7]
        )
      }
      if(my.prov=='idigbio'){
        result=as.character(
        as.idigbio(occ(query=my.species,from='idigbio',limit=1, has_coords = TRUE))[[1]]$indexTerms$taxonid
        )
      }
      if(my.prov=='inat'){
        result=as.character(
        as.inat(occ(query=my.species,from='inat',limit=1, has_coords = TRUE))[[1]]$results$taxon$id
        )
      }
      if(my.prov=='obis'){
        result=as.character(
        as.obis(occ(query=my.species,from='obis',limit=1))[[1]]$results$aphiaID
        )
      }
    }

    ,error = function(e) {an.error.occured <<- TRUE}
             
            )

    if(an.error.occured){result<-NA}
    
    spec.keys<<-c(spec.keys,ifelse(is.null(result),NA,result))
}

names(spec.keys)
    
prov.keys<-cbind(prov.keys,spec.keys)

colnames(prov.keys)[ncol(prov.keys)]<-my.prov 

}

prov.keys

Warning message in foo_ala(sources, x, y, s, z, hc, d, w, ds$ala):
“No records returned in ALA for Dreissena polymorpha”Warning message:
“Unknown or uninitialised column: `prov`.”Warning message in foo_ala(sources, x, y, s, z, hc, d, w, ds$ala):
“No records returned in ALA for Mnemiopsis leidyi”Warning message:
“Unknown or uninitialised column: `prov`.”Warning message in foo_ala(sources, x, y, s, z, hc, d, w, ds$ala):
“No records returned in ALA for Rapana venosa”Warning message:
“Unknown or uninitialised column: `prov`.”Warning message:
“bison: No records returned in Bison for Rapana venosa”Warning message:
“bison: ”Warning message:
“Unknown or uninitialised column: `prov`.”

species,ala,bison,gbif,idigbio,inat,obis
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
Carcinus maenas,284294,98734,5178595,5178595,52523,107381
Caulerpa taxifolia,83052,6974,2643172,2643172,50919,144476
Codium fragile,84082,6897,5272096,5272096,67555,145086
Dreissena polymorpha,NA,81339,2287072,2287072,116340,181566
Mnemiopsis leidyi,NA,53917,2501248,2501248,180788,106401
Pterois volitans,189453,166883,2334438,2334438,47280,159559
Rapana venosa,NA,NA,4363583,NA,370913,140416


In [43]:
# idigbio not showing Rapana venosa !!
x=occ(query='Pterois volitans',from='idigbio',limit=1, has_coords = TRUE)

In [44]:
as.idigbio(x)[[1]]$indexTerms$taxonid

[1] "2334438"

In [ ]:
########################################################################################
########################################################################################
# save outputs

objects()
save(list=objects(),file=fName )           

save(list=c('df.ALL','species.list','fao.shp','eez.shp','prov.list','prov.keys'),file=paste('short-',fName,sep=''))